## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-12-17-13-53 +0000,bbc,'Vast majority' of parents should be involved ...,https://www.bbc.com/news/articles/c2k809vnx0yo...
1,2026-02-12-17-12-21 +0000,bbc,Sir Chris Wormald stands down as head of Civil...,https://www.bbc.com/news/articles/c3wlqy6695do...
2,2026-02-12-17-09-10 +0000,nyt,"NATO Remains Vital to U.S. Security, Ex-Ambass...",https://www.nytimes.com/2026/02/12/world/europ...
3,2026-02-12-17-06-38 +0000,nyt,Immigration Officials Face Senate Panel,https://www.nytimes.com/2026/02/12/us/politics...
4,2026-02-12-17-05-24 +0000,nyt,"Crypto Super PAC Targets Al Green, a Texas Dem...",https://www.nytimes.com/2026/02/12/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2426/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
89,trump,42
228,epstein,22
194,new,16
559,he,15
65,canada,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
122,2026-02-12-12-06-00 +0000,wsj,California Gov. Gavin Newsom is on the road ag...,https://www.wsj.com/politics/gavin-newsom-the-...,137
69,2026-02-12-15-26-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...,120
227,2026-02-12-00-44-00 +0000,wsj,The GOP-led House passed a resolution designed...,https://www.wsj.com/politics/policy/gop-led-ho...,115
242,2026-02-11-23-37-50 +0000,nypost,Six House Republicans break with White House t...,https://nypost.com/2026/02/11/us-news/six-hous...,95
78,2026-02-12-15-00-00 +0000,wsj,Josh Gruenbaum’s brash ways as a federal agenc...,https://www.wsj.com/politics/national-security...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
122,137,2026-02-12-12-06-00 +0000,wsj,California Gov. Gavin Newsom is on the road ag...,https://www.wsj.com/politics/gavin-newsom-the-...
81,72,2026-02-12-14-38-32 +0000,nypost,Woman seen in photo under disgraced ex-Prince ...,https://nypost.com/2026/02/12/us-news/woman-in...
227,70,2026-02-12-00-44-00 +0000,wsj,The GOP-led House passed a resolution designed...,https://www.wsj.com/politics/policy/gop-led-ho...
69,60,2026-02-12-15-26-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...
366,60,2026-02-11-17-27-07 +0000,nypost,Ex-Google CEO Eric Schmidt spent New Year’s wi...,https://nypost.com/2026/02/11/business/ex-goog...
343,55,2026-02-11-18-53-53 +0000,nypost,"Jesse Van Rootselaar, 18, ID’d as transgender ...",https://nypost.com/2026/02/11/world-news/tumbl...
216,48,2026-02-12-02-02-00 +0000,wsj,Demand for healthcare workers is outstripping ...,https://www.wsj.com/economy/jobs/healthcare-jo...
52,41,2026-02-12-16-00-00 +0000,nypost,"LA’s top cop warns city needs 3,300 more offic...",https://nypost.com/2026/02/12/us-news/lapd-chi...
48,39,2026-02-12-16-06-28 +0000,nypost,Savannah Guthrie heartbreakingly posts old hom...,https://nypost.com/2026/02/12/us-news/savannah...
225,38,2026-02-12-00-55-36 +0000,nypost,DHS slams ‘activist judge’ for releasing four ...,https://nypost.com/2026/02/11/us-news/dhs-slam...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
